### Treehouse Outlier

See which cancer related genes in a specific prospective Treehouse sample are expressed above the 75th percentile against the entire v2 cohort.

Xena exposes a flexible query interface allowing you to select what you want as well as the shape of what you want.

A higher level python library with examples can be found at:

https://github.com/ucscXena/ucsc-xena-server/tree/master/python

Each hub also exposes a console that you can use to develop and test queries directly:

http://toil.xenahubs.net/console.html

Paste:

(query {:select [:name] :from [:dataset] :where [:like :name "%target%"]})

to get a list of all the datasets with 'target' in their title.

In [1]:
import numpy as np
import pandas as pd
import xena_query as xena

In [2]:
# Get a list of cohorts for this hub
hub = "https://xena.scellucsc.net"
dataset = "treehouse_v2_expression"
print "Cohorts:", xena.all_cohorts(hub)
print "Treehouse Datasets:", xena.datasets_list_in_cohort(hub, "treehouse_v2")

Cohorts: [u'kriegsteinRadialGliaStudy1', u'treehouse_v2', u'quakeBrainGeo1']
Treehouse Datasets: [u'treehouse_v2_clinical', u'treehouse_v2_expression']


In [3]:
# Get a list of all the features in the expression dataset
all_features = pd.DataFrame(xena.dataset_field(hub, dataset), columns=["Gene Symbol"])
print "Total number of features:", len(all_features)
all_features.head()

Total number of features: 27166


,Gene Symbol
0,A1BG
1,A1BG-AS1
2,A1CF
3,A2M
4,A2M-AS1


In [4]:
# Get a list of cancer genes as features
cancer_genes = pd.read_table("cancer_genes.tsv")
print "Number of Cancer Genes:", len(cancer_genes)
cancer_genes.head()

Number of Cancer Genes: 602


,Gene Symbol,Name,Entrez GeneId,Genome Location,Chr Band,Somatic,Germline,Tumour Types(Somatic),Tumour Types(Germline),Cancer Syndrome,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms
0,ABI1,abl-interactor 1,10006,10:26748570-26860863,10p11.2,yes,NaN,AML,NaN,NaN,L,Dom,TSG,T,KMT2A,NaN,NaN,"ABI1,E3B1,ABI-1,SSH3BP1,10006"
1,ABL1,v-abl Abelson murine leukemia viral oncogene h...,25,9:130835447-130885683,9q34.1,yes,NaN,"CML, ALL, T-ALL",NaN,NaN,L,Dom,oncogene,"T, Mis","BCR, ETV6, NUP214",NaN,NaN,"ABL1,p150,ABL,c-ABL,JTK7,bcr/abl,v-abl,P00519,..."
2,ABL2,"c-abl oncogene 2, non-receptor tyrosine kinase",27,1:179107718-179143044,1q24-q25,yes,NaN,AML,NaN,NaN,L,Dom,oncogene,T,ETV6,NaN,NaN,"ABL2,ARG,RP11-177A2_3,ABLL,P42684,ENSG00000143..."
3,ACKR3,atypical chemokine receptor 3,57007,2:-,2q37.3,yes,NaN,lipoma,NaN,NaN,M,Dom,oncogene,T,HMGA2,NaN,NaN,NaN
4,ACSL3,acyl-CoA synthetase long-chain family member 3,2181,2:222908773-222941654,2q36,yes,NaN,prostate,NaN,NaN,E,Dom,NaN,T,ETV1,NaN,NaN,"2181,PRO2194,ACS3,FACL3,O95573,ENSG00000123983..."


In [5]:
# Subset the features to just those from the cancer list
filtered_features = cancer_genes.merge(all_features, how="inner", on="Gene Symbol")
print "Cancer gene features in dataset:", len(filtered_features)
filtered_features.head()

Cancer gene features in dataset: 417


,Gene Symbol,Name,Entrez GeneId,Genome Location,Chr Band,Somatic,Germline,Tumour Types(Somatic),Tumour Types(Germline),Cancer Syndrome,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms
0,ABL2,"c-abl oncogene 2, non-receptor tyrosine kinase",27,1:179107718-179143044,1q24-q25,yes,NaN,AML,NaN,NaN,L,Dom,oncogene,T,ETV6,NaN,NaN,"ABL2,ARG,RP11-177A2_3,ABLL,P42684,ENSG00000143..."
1,ACKR3,atypical chemokine receptor 3,57007,2:-,2q37.3,yes,NaN,lipoma,NaN,NaN,M,Dom,oncogene,T,HMGA2,NaN,NaN,NaN
2,ACSL3,acyl-CoA synthetase long-chain family member 3,2181,2:222908773-222941654,2q36,yes,NaN,prostate,NaN,NaN,E,Dom,NaN,T,ETV1,NaN,NaN,"2181,PRO2194,ACS3,FACL3,O95573,ENSG00000123983..."
3,ACSL6,acyl-CoA synthetase long-chain family member 6,23305,5:131954234-132011553,5q31.1,yes,NaN,"AML, AEL",NaN,NaN,L,Dom,NaN,T,ETV6,NaN,NaN,"23305,FACL6,LACS2,ACS2,LACS5,KIAA0837,ENSG0000..."
4,ACVR1,"activin A receptor, type I",90,2:157737531-157799493,2q23-q24,yes,NaN,DIPG,NaN,NaN,O,Dom,oncogene,Mis,NaN,yes,Fibrodysplasia ossificans progressiva,"90,SKR1,ACTRI,ALK2,FOP,ACVRLK2,Q04771,ENSG0000..."


In [6]:
# Get a list of all the samples identifiers
samples = pd.DataFrame(xena.dataset_samples(hub, dataset), columns=["Sample ID"])
print "Samples found:", len(samples)
samples.head()

Samples found: 10818


,Sample ID
0,icgc/_EGAR00001415737_RNA_PAIRED_ICGC_GBM15_tu...
1,icgc/_EGAR00001415749_RNA_PAIRED_ICGC_GBM24_tu...
2,icgc/_EGAR00001415750_RNA_PAIRED_ICGC_GBM25_tu...
3,icgc/_EGAR00001415751_RNA_PAIRED_ICGC_GBM26_tu...
4,icgc/_EGAR00001415752_RNA_PAIRED_ICGC_GBM27_tu...


In [7]:
# Get expression levels for the filtered gene list for all the samples
expression = pd.DataFrame(xena.dataset_probe_values(hub, dataset, 
                                       list(samples["Sample ID"].values), 
                                       list(filtered_features["Gene Symbol"].values)),
                          index=list(filtered_features["Gene Symbol"].values),
                          columns=list(samples["Sample ID"].values), dtype=np.float32)
expression.head()

,icgc/_EGAR00001415737_RNA_PAIRED_ICGC_GBM15_tumor_SN935_0182_B_C2UKHACXXs_131105_1,icgc/_EGAR00001415749_RNA_PAIRED_ICGC_GBM24_tumor_SN935_0169_B_D2335ACXXs_130809_5,icgc/_EGAR00001415750_RNA_PAIRED_ICGC_GBM25_tumor_SN935_0169_B_D2335ACXXs_130809_6,icgc/_EGAR00001415751_RNA_PAIRED_ICGC_GBM26_tumor_SN935_0169_B_D2335ACXXs_130809_7,icgc/_EGAR00001415752_RNA_PAIRED_ICGC_GBM27_tumor_SN935_0182_B_C2UKHACXXs_131105_2,icgc/_EGAR00001415753_RNA_PAIRED_ICGC_GBM28_tumor_SN935_0182_B_C2UKHACXXs_131105_3,icgc/_EGAR00001415754_RNA_PAIRED_ICGC_GBM32_tumor_SN935_0182_B_C2UKHACXXs_131105_5,icgc/_EGAR00001415755_RNA_PAIRED_ICGC_GBM33_tumor_SN935_0182_B_C2UKHACXXs_131105_6,icgc/_EGAR00001415756_RNA_PAIRED_ICGC_GBM34_tumor_SN935_0182_B_C2UKHACXXs_131105_7,icgc/_EGAR00001415757_RNA_PAIRED_ICGC_GBM36_tumor_SN935_0182_B_C2UKHACXXs_131105_8,...,icgc/_EGAZ00001000218_81MAGABXX_3_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000219_81MAGABXX_4_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000221_81MAGABXX_6_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000223_81MAGABXX_8_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000226_81MAGABXX_1_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000227_81MAGABXX_2_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000232_81MK3ABXX_3_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000233_81MK3ABXX_4_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000236_81MK3ABXX_7_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000237_81MK3ABXX_8_withJunctionsOnGenome_dupsFlagged
ABL2,12.52810,13.10170,12.79090,12.74930,12.4276,13.74050,12.7222,12.72740,11.88170,12.88490,...,12.21010,12.31560,11.94210,13.52590,11.65350,11.55980,11.37830,12.92460,11.14700,11.92000
ACKR3,11.37600,10.21400,10.93630,10.12630,10.1251,12.85470,10.9015,10.87020,9.32999,10.50030,...,6.98112,6.85113,9.49462,12.73920,6.45319,6.72364,6.09173,6.27627,7.69791,6.54832
ACSL3,12.95930,13.87100,13.43120,14.08850,12.1520,13.47530,12.4944,14.07980,16.28190,14.29230,...,12.13600,12.17340,12.41630,11.66310,11.49090,12.70170,11.53230,11.87990,11.46780,11.20930
ACSL6,9.06482,6.33572,9.63731,6.73563,8.2234,9.74424,7.5038,7.09675,8.02462,4.80642,...,8.94622,6.07243,6.26945,4.03185,4.07634,8.57807,6.22102,3.98123,4.75369,5.67579
ACVR1,10.19550,10.67000,10.64940,10.93880,10.7445,11.15610,10.5862,11.02950,10.38810,10.16760,...,10.96490,11.43380,10.75740,11.64300,11.80290,11.37560,12.27050,12.31350,9.68646,10.92860


In [8]:
# See if any are not expressed at all in all samples
expression[(expression.T == 0).all()]

,icgc/_EGAR00001415737_RNA_PAIRED_ICGC_GBM15_tumor_SN935_0182_B_C2UKHACXXs_131105_1,icgc/_EGAR00001415749_RNA_PAIRED_ICGC_GBM24_tumor_SN935_0169_B_D2335ACXXs_130809_5,icgc/_EGAR00001415750_RNA_PAIRED_ICGC_GBM25_tumor_SN935_0169_B_D2335ACXXs_130809_6,icgc/_EGAR00001415751_RNA_PAIRED_ICGC_GBM26_tumor_SN935_0169_B_D2335ACXXs_130809_7,icgc/_EGAR00001415752_RNA_PAIRED_ICGC_GBM27_tumor_SN935_0182_B_C2UKHACXXs_131105_2,icgc/_EGAR00001415753_RNA_PAIRED_ICGC_GBM28_tumor_SN935_0182_B_C2UKHACXXs_131105_3,icgc/_EGAR00001415754_RNA_PAIRED_ICGC_GBM32_tumor_SN935_0182_B_C2UKHACXXs_131105_5,icgc/_EGAR00001415755_RNA_PAIRED_ICGC_GBM33_tumor_SN935_0182_B_C2UKHACXXs_131105_6,icgc/_EGAR00001415756_RNA_PAIRED_ICGC_GBM34_tumor_SN935_0182_B_C2UKHACXXs_131105_7,icgc/_EGAR00001415757_RNA_PAIRED_ICGC_GBM36_tumor_SN935_0182_B_C2UKHACXXs_131105_8,...,icgc/_EGAZ00001000218_81MAGABXX_3_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000219_81MAGABXX_4_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000221_81MAGABXX_6_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000223_81MAGABXX_8_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000226_81MAGABXX_1_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000227_81MAGABXX_2_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000232_81MK3ABXX_3_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000233_81MK3ABXX_4_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000236_81MK3ABXX_7_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000237_81MK3ABXX_8_withJunctionsOnGenome_dupsFlagged


In [10]:
# Calculate some stats
expression.describe()

,icgc/_EGAR00001415737_RNA_PAIRED_ICGC_GBM15_tumor_SN935_0182_B_C2UKHACXXs_131105_1,icgc/_EGAR00001415749_RNA_PAIRED_ICGC_GBM24_tumor_SN935_0169_B_D2335ACXXs_130809_5,icgc/_EGAR00001415750_RNA_PAIRED_ICGC_GBM25_tumor_SN935_0169_B_D2335ACXXs_130809_6,icgc/_EGAR00001415751_RNA_PAIRED_ICGC_GBM26_tumor_SN935_0169_B_D2335ACXXs_130809_7,icgc/_EGAR00001415752_RNA_PAIRED_ICGC_GBM27_tumor_SN935_0182_B_C2UKHACXXs_131105_2,icgc/_EGAR00001415753_RNA_PAIRED_ICGC_GBM28_tumor_SN935_0182_B_C2UKHACXXs_131105_3,icgc/_EGAR00001415754_RNA_PAIRED_ICGC_GBM32_tumor_SN935_0182_B_C2UKHACXXs_131105_5,icgc/_EGAR00001415755_RNA_PAIRED_ICGC_GBM33_tumor_SN935_0182_B_C2UKHACXXs_131105_6,icgc/_EGAR00001415756_RNA_PAIRED_ICGC_GBM34_tumor_SN935_0182_B_C2UKHACXXs_131105_7,icgc/_EGAR00001415757_RNA_PAIRED_ICGC_GBM36_tumor_SN935_0182_B_C2UKHACXXs_131105_8,...,icgc/_EGAZ00001000218_81MAGABXX_3_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000219_81MAGABXX_4_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000221_81MAGABXX_6_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000223_81MAGABXX_8_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000226_81MAGABXX_1_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000227_81MAGABXX_2_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000232_81MK3ABXX_3_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000233_81MK3ABXX_4_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000236_81MK3ABXX_7_withJunctionsOnGenome_dupsFlagged,icgc/_EGAZ00001000237_81MK3ABXX_8_withJunctionsOnGenome_dupsFlagged
count,417.000000,417.000000,417.000000,417.000000,417.000000,417.000000,417.000000,417.000000,417.000000,417.000000,...,417.000000,417.000000,417.000000,417.000000,417.000000,417.000000,417.000000,417.000000,417.000000,417.000000
mean,10.182821,10.027308,9.862326,9.939827,9.829427,11.430959,10.370827,10.001285,9.799645,9.787988,...,8.856701,8.808331,8.675457,9.121507,9.101644,9.047162,8.465183,8.703826,9.001608,8.950782
std,3.394402,3.840125,3.971373,3.930997,3.773202,3.146525,3.309657,3.916850,3.982865,3.993498,...,3.846623,3.789008,3.745486,3.600406,3.774234,3.664901,3.808564,3.896501,3.385404,3.809129
min,0.000000,0.000000,0.000000,0.000000,0.000000,2.869240,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.349130,8.526050,8.188340,8.302560,8.141290,9.074620,8.768420,8.433690,7.995030,7.843910,...,6.449660,6.458980,6.432390,6.969060,6.643780,6.841210,5.811610,6.276270,6.973770,6.548320
50%,10.917700,11.119700,10.897700,10.917000,10.855100,12.063800,11.148000,11.092700,10.733500,10.752500,...,9.768940,9.752880,9.491130,9.850690,10.348100,9.700210,9.213780,9.486780,9.790240,9.858230
75%,12.628400,12.853900,12.730500,12.812200,12.462300,13.907100,12.735600,12.727400,12.558900,12.771100,...,11.911700,11.833100,11.646300,11.786800,11.901500,11.897800,11.498200,11.769300,11.467800,11.798800
max,16.773199,16.091200,16.079901,17.346701,15.533300,17.895500,16.111700,16.346001,16.908800,17.239599,...,15.275700,16.618500,15.642200,16.379601,16.300501,15.791000,15.642300,17.018499,16.105400,16.271700


In [13]:
# Get some stats per gene
expression.T.describe().T.head()

,count,mean,std,min,25%,50%,75%,max
ABL2,10818.0,10.559373,0.894061,0.0,9.949900,10.52025,11.109875,14.506200
ACKR3,10818.0,9.568782,1.875077,0.0,8.460800,9.58020,10.764450,15.579900
ACSL3,10818.0,11.750946,0.983940,0.0,11.126225,11.68350,12.320125,16.281900
ACSL6,10818.0,0.865434,1.488314,0.0,0.000000,0.00000,1.587600,11.079700
ACVR1,10818.0,10.384686,0.964976,0.0,9.895025,10.44580,10.953450,16.419399


In [18]:
# Find all the genes in a sample (N of 1) expressed over the 75% of the cohort
sample = expression["ckcc/TH03_0016_S01"]
cutoff = expression.T.describe().T["75%"]
up = sample[(sample > cutoff)]
print "Outliers:", len(up)
up.head()

Outliers: 136


ABL2     11.47810
ACSL3    12.49850
ACSL6     7.11436
ACVR1    11.31170
AFF3     11.28830
Name: ckcc/TH03_0016_S01, dtype: float32